In [1]:
soap_notes_fp = "sample_output.csv"
note_id = "10000032-DS-22"

In [2]:
# Parameters
soap_notes_fp = "soap_note_sample1test_10415772-DS-14.csv"
note_id = "10415772-DS-14"


In [3]:
print("soap_notes_fp =", soap_notes_fp, "note_id =", note_id)

soap_notes_fp = soap_note_sample1test_10415772-DS-14.csv note_id = 10415772-DS-14


# SOAP Note Generation

This notebook performs step 1 of the Brief Hospital Course pipeline, in which we generate a SOAP note for each service based on a list of information that has been passed into a GPT-3.5 model. 

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import ast
import numpy as np

from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

import os
import openai

In [6]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [29]:
from preprocessing import *
from prompt_functions import get_replacement_data

In [8]:
root_path = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/"

## Read in Prompts

In [9]:
prompts = pd.read_csv("soap_note_prompts.csv")

## Read in Radiology Reports

In [10]:
radiology = pd.read_csv(f"{root_path}public/train/radiology.csv.gz")

## Read in Structured Data

In [311]:
###### 
# Challenge Data

# discharge summaries
discharges = pd.read_csv(f"{root_path}public/train/discharge.csv.gz")

# ed stays
train_edstays = pd.read_csv(f'{root_path}public/train/edstays.csv.gz')
valid_edstays = pd.read_csv(f'{root_path}public/valid/edstays.csv.gz')
test_edstays = pd.read_csv(f'{root_path}public/test_phase_1/edstays.csv.gz')

# triage
train_triage = pd.read_csv(f'{root_path}public/train/triage.csv.gz')
valid_triage = pd.read_csv(f'{root_path}public/valid/triage.csv.gz')
test_triage = pd.read_csv(f'{root_path}public/test_phase_1/triage.csv.gz')


In [315]:
train_edstays['split'] = "train"
valid_edstays['split'] = "valid"
test_edstays['split'] = "test"


train_triage['split'] = "train"
valid_triage['split'] = "valid"
test_triage['split'] = "test"

In [316]:
edstays = pd.concat([train_edstays, valid_edstays, test_edstays], axis=0)
triage = pd.concat([train_triage, valid_triage, test_triage], axis=0)


In [11]:

###### 
# MIMIC-IV Data

# ward transfers
transfers = pd.read_pickle(f'{root_path}mimiciv/hosp/cohort_transfers.pkl')

# higher-level services (ICU, CARD, etc)
services = pd.read_pickle(f'{root_path}mimiciv/hosp/cohort_services.pkl')

# get patient info
pts = pd.read_pickle(f'{root_path}mimiciv/hosp/cohort_patients.pkl')

# admission demographics
admissions = pd.read_pickle(f'{root_path}mimiciv/hosp/cohort_admissions.pkl')

# procedures
procs = pd.read_pickle(f'{root_path}mimiciv/hosp/cohort_procedures_icd.pkl')

# diagnoses
diags = pd.read_pickle(f'{root_path}mimiciv/hosp/cohort_diagnoses_icd.pkl')

# meds
med_orders = pd.read_pickle(f'{root_path}mimiciv/hosp/cohort_emar.pkl')


In [12]:
prescriptions = pd.read_pickle(f'{root_path}mimiciv/hosp/cohort_prescriptions.pkl')
labs = pd.read_pickle(f'{root_path}mimiciv/hosp/cohort_labevents.pkl')
microbio = pd.read_pickle(f'{root_path}mimiciv/hosp/cohort_microbiologyevents.pkl')

### Clean up/type cast data

In [13]:
transfers.loc['hadm_id'] = pd.to_numeric(transfers["hadm_id"], errors="ignore")

/tmp/ipykernel_334481/85971565.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  transfers.loc['hadm_id'] = pd.to_numeric(transfers["hadm_id"], errors="ignore")


In [14]:
med_orders = med_orders[~med_orders['charttime'].isna() & 
                        ~med_orders['medication'].isna() & 
                        ~med_orders['event_txt'].isna()]

In [15]:
procs = procs.astype({"chartdate":"datetime64[ns]"})
med_orders = med_orders.astype({"charttime":"datetime64[ns]",
                  "scheduletime":"datetime64[ns]",
                  "storetime":"datetime64[ns]",})

discharges = discharges.astype({"charttime":"datetime64[ns]",
                               "storetime":"datetime64[ns]"})

In [16]:
edstays = edstays.astype({"intime":"datetime64[ns]",
                         "outtime":"datetime64[ns]"})

In [17]:
prescriptions = prescriptions.astype({"starttime":"datetime64[ns]",
                      "stoptime":"datetime64[ns]"})
labs = labs.astype({"charttime":"datetime64[ns]",
                      "storetime":"datetime64[ns]"})

microbio = microbio.astype({"chartdate":"datetime64[ns]",
                      "charttime":"datetime64[ns]",
                           "storetime":"datetime64[ns]",
                      "charttime":"datetime64[ns]"})

In [195]:
# generate free text sections of tables concatenating columns
med_orders['admin_text'] = med_orders['medication'].fillna("") + " at " + med_orders['charttime'].dt.strftime('%B %d, %Y, %r').fillna("")
# TODO SORAYA: Add the right information in here that we actually want for the meds
prescriptions['drug'] = prescriptions['drug'].str.lower()
Flush = [".*flush.*", ".*dextrose.*", ".*sodium chloride.*", ".*syringe.*", ".*d10w.*",
        ".*nacl.*", ".*d5.*",".*solution.*",  ".*water.*", "lr ",".*bag.*", ".*soln.*",
        ".*dextran.*", ".*saline.*",".*ringer.*", r"\bns\b", r"\blr\b", ".*sw.*"]

Other = [".*vaccine.*", ".*rinse.*", ".*vial.*", ".*tear.*", ".*nicotine.*", ".*iohexol.*", ".*ethanol.*",
        ".*spray.*", ".*patch.*", ".*barium.*", ".*catheter.*", ".*pad.*"]
Lotions = [".*oil.*",".*cream.*", ".*oint.*", ".*gel.*",".*lotion.*"]
GI = [".*ondansetron.*", ".*prazol.*",".*senna.*",".*docusate.*", ".*scopolamine.*", ".*ibuprofen.*", 
      ".*lactulose.*", ".*simethicone.*",  ".*bisacodyl.*", ".*polyethylene.*", ".*ranitidine.*", ".*lactulose.*",
     ".*magnesium.*", ".*famotidine.*", ".*magnesia.*", ".*dicyclomin.*", ".*promethazine.*", ".*zofran.*"]
Pain = [".*caine.*", ".*acetaminophen.*", ".*codein.*", ".*gabapentin.*"]
Mineral = [".*vitamin.*",  ".*folic acid.*", ".*ferrous.*", ".*thiamine.*", ".*neutra-phos.*"]
SoreThroat = [".*cepacol.*", ".*guaifenesin.*"]
Primary = [".*statin.*"]
Antidepress = [".*sertraline.*", ".*bupropion.*", ".*xetine.*", ".*pram.*", ".*fluvoxamine.*", 
          ".*faxine.*", ".*levomilnacipran .*", ".*mirtazapine.*", ".*triptyline.*", ".*ipramine.*"]
Sleeping = [".*trazodone.*", ".*zolpidem.*", ".*zaleplon.*", ".*triazolam .*", ".*zaleplon.*", 
           ".*temazepam.*", ".*suvorexant.*", ".*ramelteon.*", ".*lemborexant.*", ".*eszopiclone.*",
           ".doxepin.*", ".*daridorexant.*", ".*zz.*"]
dementia = ['.*donepezil.*', '.*rivastigmine.*', '.*galantamine.*', ".*memantine.*"]
MED_check = Flush + Other + Lotions + Pain + Mineral + SoreThroat + Primary + Sleeping + GI + dementia + Antidepress

# Creating a single regex pattern
combined_pattern = '|'.join(MED_check)

# Filter rows based on regex pattern
mask = prescriptions['drug'].notna() & prescriptions['drug'].str.contains(combined_pattern, regex=True)
prescriptions = prescriptions[~mask]

prescriptions['text'] = prescriptions['drug'].fillna("") + " " + prescriptions['prod_strength'].fillna("")

In [58]:
# TODO THOMAS: figure out how to better encapsulate lab values

# Filtering labs for only abnormal lab values
filtered_labs = labs[(labs['valuenum'] < labs['ref_range_lower']) | (labs['valuenum'] > labs['ref_range_upper'])]

filtered_labs['text'] = (filtered_labs['category'].fillna("") + " ordered with "+filtered_labs['priority'].fillna("")+" priority, " + filtered_labs['fluid'].fillna("") + " " 
                + filtered_labs['label'].fillna("") + ": " + filtered_labs['value'].fillna("") + " " 
                + filtered_labs['valueuom'].fillna("")
                + " with additional comments" + filtered_labs['comments'].fillna(""))

#Commented out for now to avoid overwriting original labs, but uncomment to bring the filtered labs back into the labs name
labs = filtered_labs

In [20]:
# TODO SORAYA: check for those patients that don't have comments (they might not have useful information in the test)

#drop labs with no organism detected
microbio.dropna(subset=['org_name'], inplace=True)
# Filter out rows where 'org_name' contains 'cancelled'
microbio = microbio[~microbio['org_name'].str.contains('cancelled', case=False)]

# regex pattern
pattern = r'\b(not detected|negative|nonreactive|no|discontinued|not|unknown|absent|cancelled|non|rare|unable|sparse|___|none|duplicate|invalidates)\b'

# Check for the negation patterns and eliminate them
microbio['comments'] = microbio['comments'].str.lower()
microbio['negation_or_negative_detected'] = microbio['comments'].str.contains(pattern, regex=True)
microbio['negation_or_negative_detected'] = microbio['negation_or_negative_detected'].fillna(False)
microbio = microbio[~microbio['negation_or_negative_detected']]

microbio['text'] = microbio['test_name'].fillna("") + ": " + microbio['org_name'].fillna("")

/tmp/ipykernel_334481/2317201544.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  microbio['negation_or_negative_detected'] = microbio['comments'].str.contains(pattern, regex=True)
/tmp/ipykernel_334481/2317201544.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  microbio['negation_or_negative_detected'] = microbio['negation_or_negative_detected'].fillna(False)


In [21]:
pts

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
7,10000117,F,48,2174,2008 - 2010,NaN
23,10000764,M,86,2132,2014 - 2016,NaN
30,10000935,F,52,2182,2008 - 2010,2187-11-12
33,10000980,F,73,2186,2008 - 2010,2193-08-26
...,...,...,...,...,...,...
299688,19999204,M,61,2146,2017 - 2019,2146-08-29
299689,19999287,F,71,2191,2008 - 2010,2197-09-02
299706,19999784,M,57,2119,2017 - 2019,NaN
299707,19999828,F,46,2147,2017 - 2019,NaN


## Generate Replacement Data

In case something is missing from the preceding text, we can generate it using this

In [321]:
from prompt_functions import get_replacement_data

In [208]:
# discharge summaries
target_train = pd.read_pickle(root_path + "public/train/discharge_target_with_preceding_text.pickle")
target_valid = pd.read_pickle(root_path + "public/valid/discharge_target_with_preceding_text.pickle")
target_test = pd.read_pickle(root_path + "public/test_phase_1/discharge_target_with_preceding_text.pickle")

In [294]:
z = target_test.sample(20)

In [ ]:
target_train_with_structured = get_replacement_data(target_train, edstays, 
                         pts, triage, transfers, diags, procs, prescriptions, labs, microbio)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [349]:
target_valid_with_structured = get_replacement_data(target_valid, edstays, 
                         pts, triage, transfers, diags, procs, prescriptions, labs, microbio)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700


In [ ]:
target_test_with_structured = get_replacement_data(target_test,
                                                   edstays, 
                         pts, triage, transfers, diags, procs, prescriptions, labs, microbio)


In [ ]:
target_train_with_structured.to_pickle(root_path + "public/train/discharge_target_with_preceding_text&structured_data.pickle")
target_valid_with_structured.to_pickle(root_path + "public/valid/discharge_target_with_preceding_text&structured_data.pickle")
target_test_with_structured.to_pickle(root_path + "public/test_phase_1/discharge_target_with_preceding_text&structured_data.pickle")

In [326]:
target_test_with_structured.columns

Index(['index', 'note_id', 'hadm_id', 'discharge_instructions',
       'brief_hospital_course', 'discharge_instructions_word_count',
       'brief_hospital_course_word_count', 'bhc_preceding_text', '_merge',
       'subject_id', 'note_type', 'note_seq', 'charttime', 'storetime', 'text',
       'age', 'sex', 'cc', 'diags', 'procs', 'prescriptions', 'labs',
       'microbio'],
      dtype='object')

In [336]:
from IPython.display import display, HTML

def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

In [348]:
pretty_print(target_test_with_structured.sample(1)[["age", "sex", "cc", "diags", "procs", "prescriptions", "labs", "microbio"]])

,age,sex,cc,diags,procs,prescriptions,labs,microbio
7939,Age:59,Sex:F,Chief Complaints:CHEST PAIN,"Admission Diagnoses (ordered by importance):Other chest painAtrial fibrillationOther fluid overloadSickle-cell traitAnemia in chronic kidney diseaseHypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal diseaseEnd stage renal diseaseGlycogenosisHyperparathyroidism, unspecifiedLeukocytosis, unspecified",Procedures (ordered by priority):None,"Medications (ordered chronologically):metoprolol tartrate 5mg/5mL Vialipratropium bromide neb 2.5mL Vialmetoprolol tartrate 5mg/5mL Vialvalsartan 40mg Tabletepoetin alfa 4000 Unit Vialhydroxyurea 500mg Capsuleamlodipine 5mg Tabletsevelamer carbonate 800mg Tabletheparin (hemodialysis) 1000 Units/mL- 10mL Vialheparin 5000 Units / mL- 1mL Vialdiltiazem 50mg/10mL Vialdiltiazem 50mg/10mL Vialfurosemide 40mg Tabletepoetin alfa 10,000 Unit Vialdiltiazem 30 mg Tabhydromorphone (dilaudid) 1mg/1mL Syringehydromorphone (dilaudid) 2mg Tabletcyclobenzaprine 10 mg Tablorazepam 1mg Tabletallopurinol 100mg Tabheparin (hemodialysis) 1000 Units/mL- 10mL Vialcyanocobalamin 100 mcg Tabletdoxercalciferol 2 mcg / 1 mL Vialalbuterol-ipratropium 14.7g Inhaler","Labs:Chemistry ordered with STAT priority, Blood Lactate Dehydrogenase (LD): 258 IU/L with additional commentsChemistry ordered with STAT priority, Blood Troponin T: ___ ng/mL with additional commentsCTROPNT > 0.10 NG/ML SUGGESTS ACUTE MI.Hematology ordered with ROUTINE priority, Blood Hematocrit: 17.1 % with additional commentsHematology ordered with ROUTINE priority, Blood Hemoglobin: 6.0 g/dL with additional commentsHematology ordered with ROUTINE priority, Blood MCH: 37.3 pg with additional commentsHematology ordered with ROUTINE priority, Blood MCHC: 35.1 % with additional commentsHematology ordered with ROUTINE priority, Blood MCV: 106 fL with additional commentsHematology ordered with ROUTINE priority, Blood RDW: 23.0 % with additional commentsHematology ordered with ROUTINE priority, Blood Red Blood Cells: 1.61 m/uL with additional commentsChemistry ordered with ROUTINE priority, Blood Alkaline Phosphatase: 253 IU/L with additional commentsChemistry ordered with ROUTINE priority, Blood Creatinine: 5.3 mg/dL with additional commentsChemistry ordered with ROUTINE priority, Blood Glucose: ___ mg/dL with additional commentsIF FASTING, 70-100 NORMAL, >125 PROVISIONAL DIABETES.Chemistry ordered with ROUTINE priority, Blood Lactate Dehydrogenase (LD): 260 IU/L with additional commentsChemistry ordered with ROUTINE priority, Blood Troponin T: ___ ng/mL with additional commentsCTROPNT > 0.10 NG/ML SUGGESTS ACUTE MI.Chemistry ordered with ROUTINE priority, Blood Urea Nitrogen: 29 mg/dL with additional comments",Microbiology Culture Results:None


In [331]:
target_test_with_structured.head(2)

,index,note_id,hadm_id,discharge_instructions,brief_hospital_course,discharge_instructions_word_count,brief_hospital_course_word_count,bhc_preceding_text,_merge,subject_id,...,storetime,text,age,sex,cc,diags,procs,prescriptions,labs,microbio
0,0,19766998-DS-20,26231944,"Dear Mr. ___,\n\nIt was a pleasure to care for...","___ with PMH HCV, ETOH cirrhosis with ascites,...",145,62,\nName: ___ Unit No: ___\n ...,both,19766998,...,2157-07-24 10:22:00,\nName: ___ Unit No: ___\n ...,Age:\n53\n\n,Sex:\nM\n\n,Chief Complaints:\nASCITES/NEW LIVER MASS\n\n,Admission Diagnoses (ordered by importance):\n...,Procedures (ordered by priority):\nNone\n\n,Medications (ordered chronologically):\nciprof...,"Labs:\nHematology ordered with STAT priority, ...",Microbiology Culture Results:\nNone\n\n
1,1,10336082-DS-11,28542384,"Dear Ms. ___, \n We had the pleasure of taking...",___ yo F with recent lumbar laminectomy c/b MS...,76,466,\nName: ___ Unit No: ___\n \nA...,both,10336082,...,2127-12-19 07:20:00,\nName: ___ Unit No: ___\n \nA...,Age:\n81\n\n,Sex:\nF\n\n,Chief Complaints:\nSUTURES BLEEDING\n\n,Admission Diagnoses (ordered by importance):\n...,Procedures (ordered by priority):\nNone\n\n,Medications (ordered chronologically):\ncyclob...,Labs:\nHematology ordered with ROUTINE priorit...,Microbiology Culture Results:\nNone\n\n
